In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV

In [2]:
# Importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

We will be working on the heart disease dataset.

In [3]:
# Loading the data to pandas dataframe
heart_data = pd.read_csv('datasets/heart.csv')

In [4]:
# Print first 5 rows
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
# Rows and columns
heart_data.shape

(303, 14)

In [6]:
# Checking missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [7]:
# Checking distribution of target variable

In [8]:
heart_data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

0 $\longrightarrow$ Healthy Heart  
1 $\longrightarrow$ Defective Heart

## Splitting into features and target

In [9]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [10]:
print(X)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [11]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


In [12]:
X = np.asarray(X)
Y = np.asarray(Y)

## Model Selection

Comparing the models with default hyperparameters values using Cross Validation.

In [13]:
# List of models
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [20]:
def compare_models_cross_validation():
    for model in models:
        cv_score = cross_val_score(model, X, Y, cv=5)
        mean_accuracy = np.mean(cv_score)
        mean_accuracy = mean_accuracy * 100
        mean_accuracy = round(mean_accuracy, 2)
        print(f'Cross validations accuracies for model {model}: {cv_score}')
        print(f'Accuracy score for the model {model}: {mean_accuracy}%')
        print('-' * 120)

In [21]:
compare_models_cross_validation()

Cross validations accuracies for model LogisticRegression(max_iter=1000): [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy score for the model LogisticRegression(max_iter=1000): 82.83%
------------------------------------------------------------------------------------------------------------------------
Cross validations accuracies for model SVC(kernel='linear'): [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy score for the model SVC(kernel='linear'): 82.83%
------------------------------------------------------------------------------------------------------------------------
Cross validations accuracies for model KNeighborsClassifier(): [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy score for the model KNeighborsClassifier(): 64.39%
------------------------------------------------------------------------------------------------------------------------
Cross validations accuracies for model RandomForestClassifier(random_state=0): 

Inference: For the Heart Disease dataset **Random Forest Classifier** has the best accuracy score with the default hyperparameters values.

## Comparing the models with differente hyperparameters values using GridSearchCV

In [31]:
# List of models
models_list = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [32]:
# Creating dictionary with the hyperparameters values
model_hyperparameters = {
    
    'log_reg_hyperparameters': {
        'C': [1, 5, 10, 20]
    },
    
    'svc_hyperparameters': {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [1, 5, 10, 20]
    }, 
    
    'KNN_hyperparameters': {
        'n_neighbors': [3, 5, 10]
    }, 
    
    'random_forest_hyperparameters': {
        'n_estimators': [10, 20, 50, 100]
    }
}

In [33]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters'])


In [34]:
model_keys = list(model_hyperparameters.keys())
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


## Appying GridSearchCV

In [37]:
def ModelSelection(list_of_models, hyperparameters_dictionary):
    
    # List of the best parameters
    result = []
    
    i = 0
    
    for model in list_of_models:
        key = model_keys[i]
        params = hyperparameters_dictionary[key]
        i += 1
        
        print(model)
        print(params)
        print('-' * 120)
        
        classifier = GridSearchCV(model, params, cv=5)
        
        # Fitting the data
        classifier.fit(X, Y)
        
        # Storing the result
        result.append({
            'model used': model,
            'highest score': classifier.best_score_, 
            'best hyperparameters': classifier.best_params_        
        })
        
    result_dataframe = pd.DataFrame(result, columns=['model used', 'highest score', 'best hyperparameters'])
    return result_dataframe
    

In [38]:
ModelSelection(models_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
------------------------------------------------------------------------------------------------------------------------
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
------------------------------------------------------------------------------------------------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
------------------------------------------------------------------------------------------------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
------------------------------------------------------------------------------------------------------------------------


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}


**Random Forest with n_estimators = 100 has the highest accuracy**